# Cómo encontrar dominios en un conjunto de proteínas y leer los datos en una tabla en R

La base de datos de dominos y familias proteicas Pfam se crea automáticamente mediante el uso de los programa del paquete HMMER3. Éste **no** es un paquete de R, sino que se ejecuta desde la línea de comandos o el terminal. A partir de un alineamiento *semilla* de una familia o dominio se genera (`hmmbuild`) un modelo de Markov escondido (HMM) con el que se pueden buscar (`hmmsearch`) nuevos miembros de esta misma familia en una base de datos de secuencias. También es possible realizar la búsqueda inversa: determinar si alguno de los dominios presentes en una base de datos de perfiles HMM se puede reconocer en la secuencia de una proteína (`hmmscan`).

En esta carpeta de trabajo tenéis los alineamientos *semilla* de cuatro dominios del clan ANL: ACAS_N, AMP-binding, GH3 y LuxE. También tenéis las secuencias de todas la proteínas codificadas en el genoma de la bacteria `Salinibacter ruber`. El objetivo es determinar si existen dominios del clan ANL en este genoma.

La mayor parte de la práctica se realiza en la linea de comandos, no en R. Para poder recordar todos los pasos realizados en la terminal es necesario escribirlos en un archivo de texto plano, al que podemos darle una extensión ".sh" para indicar que se trata de un *script* de BASH. Encuentra en esta carpeta el archivo `dominios.sh`, donde se han guardado todos los comandos necesarios para realizar la práctica. Brevemente:

1. Primero instalaremos `hmmer3` usando el terminal:

`conda install -c bioconda hmmer`

2. Construímos los HMM de cada dominio, siguiendo el esquema:

`hmmbuild AMP-binding.hmm AMP-binding_seed.sto`

3. Concatenamos los cuatro HMM:

`cat *.hmm > ANL_clan`
    
4. Comprimiremos e indexaremos los HMM del clan:

`hmmpress ANL_clan`
    
5. Y finalmente escaneamos los perfiles.

`hmmscan ANL_clan Salinibacter_ruber.fasta --tblout perSeq.out --domtblout perDom.out`

Los comandos anteriores pueden haberse ejecutado uno a uno en la terminal o la serie completa de una vez si estaban guardados en un script. Por ejemplo, el script `dominios.sh` producirá todos los resultados necesarios al ejecutarlo de la manera siguiente:

`bash dominos.sh`

Suponiendo que existe el archivo `perDom.out`, vamos ahora a leerlo en R mediante la función `read_domtblout()` del paquete `rhmmer`.

In [1]:
system2(command = 'hmmbuild',
        args = c('AMP-binding.hmm', 'AMP-binding_seed.sto'))
system2(command = 'hmmbuild',
        args = c('ACAS_N.hmm', 'ACAS_N_seed.sto'))
system2(command = 'hmmbuild',
        args = c('GH3.hmm', 'GH3_seed.sto'))
system2(command = 'hmmbuild',
        args = c('LuxE.hmm', 'LuxE_seed.sto'))

In [2]:
system2(command = 'cat', args = '*.hmm', stdout = 'ANL_clan')
system2(command = 'hmmpress', args = 'ANL_clan')
system2(command = 'hmmscan',
        args = c('ANL_clan',
                 'Salinibacter_ruber.fasta',
                 '--tblout', 'perSeq.out',
                 '--domtblout', 'perDom.out'))

In [ ]:
library('rhmmer')
dominios <- read_domtblout('perDom.out')
dominios

Con el simple propósito de ilustrar las ventajas de tener los resultados en la memoria de trabajo de la sesión de R, a continuación aplicamos algunas funciones para resumir e interpretar los resultados.

In [ ]:
# La función table() cuenta el número de ocurrencias de cada combinación
# posible entre los valores de las columnas seleccionadas. La función
# t() transpone la tabla para visualizarla mejor:

t(table(dominios[,c('domain_name','query_name')]))

De los cuatro dominos del clan ANL, solo tres se encuentran entre las proteínas de *Salinibacter ruber*. Entre las 2780 proteínas en esta bacteria, solo 7 tienen alguno de los dominios ANL. Curiosamente, tres de ellas tienen más de uno. Cabe preguntarse si estan separados o si alguna predicción es redundante. Es decir, puesto que son dominios relacionados: ¿podría ser que con dos HMMs diferentes (el de ACAS_N y el de AMP-binding, por ejemplo) estemos detectando un mismo segmento en alguna de las proteínas Q2S1J8 y Q2S1X8? Veamoslo.

In [ ]:
# Esta sintaxis selecciona las filas y las columnas de la tabla "dominios"
# que queremos ver:

dominios[dominios$query_name == "tr|Q2S1J8|Q2S1J8_SALRD", c('domain_name','ali_from','ali_to','domain_ievalue')]

Así pues, este fragmento de la tabla `dominios` muestra que efectivamente el HMM de ACAS_N también detecta partes del dominio AMP-binding en la proteína Q2S1J8. La gran diferencia en valores E entre el dominio AMP-binding y los tres últimos dominios ACAS_N con los que solapa nos decantan por AMP-binding.

In [ ]:
dominios[dominios$query_name == "tr|Q2S1X8|Q2S1X8_SALRD", c('domain_name','ali_from','ali_to','domain_ievalue')]

La proteína Q2S1X8 tiene la misma estructura. Pero esta vez no hay *falsos positivos*.